In [1]:
import pandas as pd
import numpy as np
import re
from langid.langid import LanguageIdentifier, model
from bs4 import BeautifulSoup
import requests

# This is a .py file where I store all my functions to be used in the code
import self_created_functions as scf 

# Data Scrapping
#### Getting the data

In total I handpicked and scrapped 28 list of books from Goodreads using a scrapy spider created by [havanagraw1](https://github.com/havanagrawal/GoodreadsScraper).
The spider grabs two different items. One on the books within the list and the other for the authors within the list. I will only be using the data on the books. More info can be found in the link above on how it works and the items scrapped.

The lists I scrapped from goodreads are:

|#|List Scrapped|
|---|:---|
|1|Best_Page_Turners_with_Redeeming_Social_Value|
|2|Couldn_t_Put_The_Book_Down_|
|3|Books_you_wish_more_people_knew_about_Part_II|
|4|Best_Books_of_the_21st_Century|
|5|Books_that_Blew_Me_Away_and_that_I_Still_Think_About_of_all_types_|
|6|Best_Unknown_but_must_be_Known_books_|
|7|1001_Books_You_Must_Read_Before_You_Die|
|8|Books_That_Everyone_Should_Read_At_Least_Once|
|9|Lesser_Known_Authors|
|10|What_To_Read_Next|
|11|The_Most_Influential_Books|
|12|100_Books_to_Read_in_a_Lifetime_Readers_Picks|
|13|Books_That_Should_Be_Made_Into_Movies|
|14|Must_Read_Non_Fiction|
|15|I_m_glad_someone_made_me_read_this_book|
|16|Best_Books_Ever|
|17|Books_With_a_Goodreads_Average_Rating_of_4_5_and_above_and_With_At_Least_100_Ratings|
|18|Books_that_Changed_the_Way_You_View_Life|
|19|100_Mysteries_and_Thrillers_to_Read_in_a_Lifetime_Readers_Picks|
|20|Read_Them_Twice_At_Least|
|21|Books_You_Wish_More_People_Knew_About|
|22|Best_Young_Adult_Books|
|23|Interesting_and_Readable_Nonfiction|
|24|Best_Books_of_the_18th_Century|
|25|Best_Books_of_the_Decade_2000s|
|26|Best_for_Book_Clubs|
|27|Best_Science_Fiction_Fantasy_Books|
|28|Best_Books_of_the_Decade_1990s|

# Data Cleaning

As the data are saved into individual json files, I first concat them together into one dataframe and save it as a csv.

In [2]:
# Grab, Load, Concat and Save raw dataframe to csv from json files
# To run code unzip the scrapped_json_file folder
books = scf.create_df_from_json_files("book")

---START---
List Loaded:  raw_datasets/scrapped_json_files/book_3187.Interesting_and_Readable_Nonfiction
Rows: 2176, Columns: 19
Data not captured from url: 0
List Loaded:  raw_datasets/scrapped_json_files/book_5.Best_Books_of_the_Decade_2000s
Rows: 7074, Columns: 19
Data not captured from url: 0
List Loaded:  raw_datasets/scrapped_json_files/book_17.Best_Books_of_the_Decade_1990s
Rows: 3024, Columns: 19
Data not captured from url: 0
List Loaded:  raw_datasets/scrapped_json_files/book_3.Best_Science_Fiction_Fantasy_Books
Rows: 7675, Columns: 18
Data not captured from url: 7668
List Loaded:  raw_datasets/scrapped_json_files/book_43.Best_Young_Adult_Books
Rows: 10000, Columns: 19
Data not captured from url: 0
List Loaded:  raw_datasets/scrapped_json_files/book_11618.Best_Page_Turners_with_Redeeming_Social_Value
Rows: 3298, Columns: 19
Data not captured from url: 0
List Loaded:  raw_datasets/scrapped_json_files/book_2994.Couldn_t_Put_The_Book_Down_
Rows: 10000, Columns: 19
Data not captur

#### In total I scrapped 175,406 books however there are a lot of null values and possibly duplicates...

In [3]:
books.head(3)

,url,title,author,num_ratings,num_reviews,avg_rating,num_pages,language,publish_date,original_publish_year,genres,awards,characters,places,isbn,isbn13,rating_histogram,series,asin
0,https://www.goodreads.com/book/show/1845.Into_...,Into the Wild,Jon Krakauer,983231.0,24367.0,4.00,215.0,English,1997-01-20 00:00:00,1996.0,"[Environment, Travel, Survival, Biography Memo...",[Washington State Book Award (1997)],[Christopher McCandless],"[Mexico, Virginia, The Slabs, Mojave Desert, C...",385486804.0,9.780385e+12,"{'5': 363210, '4': 358039, '3': 186642, '2': 4...",NaN,NaN
1,https://www.goodreads.com/book/show/168484.Fem...,Feminism Is for Everybody: Passionate Politics,bell hooks,18885.0,1586.0,4.14,123.0,English,2000-10-01 00:00:00,NaN,"[Social Movements, Politics, Sociology, Race, ...",NaN,NaN,NaN,896086283.0,9.780896e+12,"{'5': 8328, '4': 6498, '3': 2843, '2': 792, '1...",NaN,NaN
2,https://www.goodreads.com/book/show/55403.Blac...,Black Hawk Down: A Story of Modern War,Mark Bowden,59451.0,1727.0,4.28,386.0,English,1999-02-10 00:00:00,NaN,"[War, Africa, North American Hi..., Cultural, ...",[National Book Award Finalist for Nonfiction (...,NaN,[Mogadishu],871137380.0,9.780871e+12,"{'5': 28462, '4': 21639, '3': 7502, '2': 1252,...",NaN,NaN


In [4]:
# Dropping features that holds no relevance to what we want to do
unwanted_book_cols = ['asin','isbn','isbn13','characters','places','publish_date','series','rating_histogram','awards']
books_clean = books.drop(unwanted_book_cols,axis=1)

In [5]:
books_clean.dtypes

url                       object
title                     object
author                    object
num_ratings              float64
num_reviews              float64
avg_rating               float64
num_pages                float64
language                  object
original_publish_year    float64
genres                    object
dtype: object

### Duplicate Items
The different lists might contain the same books.

In [6]:
# Num of duplicates
books_clean['title'].duplicated().sum()

108468

In [7]:
# Drop duplicates
books_clean.drop_duplicates(subset=['title'],keep='first',inplace=True)
books_clean['title'].duplicated().sum()

0

In [8]:
print(f'After dropping duplicates we are left with: {books_clean.shape[0]}, books!')

After dropping duplicates we are left with: 66938, books!


### Null Values

In [9]:
# There's a row that except for the url are null
print(books_clean.isnull().sum())
books_clean[books_clean.title.isnull()].tail(3)

url                          0
title                        1
author                       1
num_ratings                  1
num_reviews                  1
avg_rating                   1
num_pages                 4490
language                  6380
original_publish_year    31383
genres                    9865
dtype: int64


,url,title,author,num_ratings,num_reviews,avg_rating,num_pages,language,original_publish_year,genres
12281,https://www.goodreads.com/book/show/25499718-c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Looking at the above null rows and matching it with the data not captured from url when loading the json file [here](#Data-Cleaning), it looks like the spider is unable to scrape the lists: "Books_That_Should_Be_Made_Into_Movies", "Best_Science_Fiction_Fantasy_Books", and "Best_for_Book_Clubs".

In [10]:
# dropping all rows with title as blank
books_clean = books_clean[books_clean.title.notna()]

In [11]:
books_clean.isnull().sum()

url                          0
title                        0
author                       0
num_ratings                  0
num_reviews                  0
avg_rating                   0
num_pages                 4489
language                  6379
original_publish_year    31382
genres                    9864
dtype: int64

To fill in the remaining null values I shall try scrapping from wikipedia's info box using [wptools](https://pypi.org/project/wptools/)
![](./images/infobox.png)

In [12]:
# Scrape wikipedia info-box for null values, clean the data and save to csv
# Run code, type "Y" to run function or any other letters to skip
scf.wptools_scrape_save_tocsv(df = books_clean,save_to ="./raw_datasets/missing_info.csv")

Input Y to run, or any other letters to skip: n


In [13]:
#Load the scrapped wikipedia - csv file scrapped above
missing_info = pd.read_csv("./raw_datasets/missing_info.csv")
#Keep only features wanted
missing_info = missing_info[['name','author','language','pages','genre','pub_date']]
print('Shape:',missing_info.shape)
print('Null Values:')
print(missing_info.isnull().sum())
missing_info.head(3)

Shape: (46, 6)
Null Values:
name         0
author       0
language     5
pages        1
genre        5
pub_date    25
dtype: int64


,name,author,language,pages,genre,pub_date
0,The October Horse,[[Colleen McCullough]],English,608.0,"Historical, novel",NaN
1,Germany: Memories of a Nation,[[Neil MacGregor]],English,598.0,NaN,2014
2,Darwin's Radio,[[Greg Bear]],English,448.0,"Science, fiction, novel",NaN


It looks like wikipedia's info-box isn't as complete as I wanted and the maximum possible books with missing info I can plug in is 46. Nevertheless I shall plug in as much as possible.

In [14]:
#change the name of the missing_info columns to match the main df books
missing_info.rename(columns={'name':'title',
                             'pages':'num_pages',
                             'genre':'genres',
                             'pub_date':'original_publish_year'
                            },inplace=True)

In [15]:
# Comparing two dataframes, replace the reference dataframe
# with the main dataframes index so that I can use combine_first
# to fill in the null value for the main dataframe
scf.index_conversion(books_clean,missing_info,'title','num_pages')
df = books_clean.combine_first(missing_info)

print('Total Null values filled in',books_clean.isnull().sum().sum()-df.isnull().sum().sum())

Total Null values filled in 52


Next we shall try using the library [langid](https://github.com/saffsd/langid.py) to detect the language of the titles of books and fill in the null value for the column language.

langid returns the iso 69-1 codes of the different languages. to get the language itself, we need to reference the table from wikipedia

steps taken
1. Import langid and use it to get a list of languages referencing the title of the books
2. Scrape wikipedia ISO codes page and extract the table of codes
3. Map the codes to the actual language
4. Fill in the null values in the dataframe

In [16]:
# Get a list of titles with language as null
lang_null = list(df['title'][df.language.isnull()])

# set function to check title's language
identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)

# Append to a list
lang=[identifier.classify(title)[0] for title in lang_null]
print('languages detected:',set(lang))

languages detected: {'jv', 'ar', 'fi', 'mg', 'tl', 'hi', 'cy', 'ms', 'be', 'eu', 'sl', 'da', 'ko', 'pt', 'it', 'sq', 'cs', 'no', 'eo', 'hu', 'mr', 'et', 'ca', 'la', 'ro', 'xh', 'ps', 'bg', 'az', 'vi', 'de', 'sk', 'uk', 'fr', 'sv', 'es', 'el', 'mt', 'ga', 'id', 'hr', 'tr', 'fa', 'nl', 'pl', 'en', 'lt'}


In [17]:
# Extract the language table for the 639-1 codes from wikipedia
iso_url = "https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes"
response = requests.get(iso_url)
print('Response',response.status_code)
soup = BeautifulSoup(response.text,'html.parser')
table = soup.find('table',{'class':"wikitable"})
iso = pd.read_html(str(table))
iso_df = pd.DataFrame(iso[0])
iso_df.head(3)

Response 200


,ISO language name,639-1,639-2/T,639-2/B,639-3,Notes
0,Abkhazian,ab,abk,abk,abk,also known as Abkhaz
1,Afar,aa,aar,aar,aar,NaN
2,Afrikaans,af,afr,afr,afr,NaN


In [18]:
# Map the 639-1 code to the ISO language name
lang = [iso_df['ISO language name'][iso_df[iso_df['639-1']==x].index[0]] for x in lang]

# Fill in the null values..
lang_null_index = list(df['title'][df.language.isnull()].index)
for i,x in zip(lang_null_index,lang):
    df.loc[i,'language'] = x

For the reminding missing values, due to time constraint I shall not try to scrape any further. Instead I shall drop the column original_publish_year, as more then half of the book's publish year is missing. Then I shall drop the remaining books that still have a null value.

In [19]:
df.drop(columns=['original_publish_year'],axis=1,inplace=True)
df.dropna(inplace=True)
print(f"We're left with {len(df)} books in our dataframe")

We're left with 54478 books in our dataframe


### Cleaning the column genres
Genres are strings within a list. Some of the genres are combined together eg. SurvivalFiction while others are split apart Non fiction. some are plural etc.

Example: ['Sports', 'Journalism', 'Soccer', 'History', 'Sociology', 'Football', 'British Literature', 'Writing', 'European Literature', 'Crime', 'Autobiography', 'TrueCrime', 'Memoir', 'Nonfiction']

In [20]:
# Remove all non alphabetical characters
df['genres']=df['genres'].apply(lambda x: re.sub('[^A-Za-z]+', ' ', str(x)))

# Remove front and back spacing and lowercase all
df['genres']=df['genres'].apply(lambda x: x.strip().lower())

# Convert back to a list
df['genres'] = df['genres'].apply(lambda x: x.split())

#Lemmatize
df = scf.lemma_column(df,'genres')

# Convert to str
df['genres'] = df['genres'].apply(lambda x: ' '.join(ele for ele in x))

#remove stopwords
df = scf.remove_stopwords(df,'genres')

# Drop the "s" at the end of every word
df['genres'] = df['genres'].apply(lambda x: ' '.join(ele[:-1] if ele[-1] == 's' else ele for ele in x.split()))

print('Total Number of Genres and subGenres: ',len(set(scf.flatten([x.split() for x in [x for x in df['genres']]]))))

Total Number of Genres and subGenres:  891


Each book's list of genres can be as short as 1 and as long as 202. Without manually going into each book and editing the genre I would not be able to get a standardise set of genres to use as a feature for my recommender system (example, fiction vs nonfiction). As such I shall leave it as it is for now... 

### Cleaning the column author

In [21]:
df.reset_index(drop=True,inplace=True)

In [22]:
#Remove punctuations
#cannot use regex here as there're authors in other languages
to_replace = [".",",","/","'"]
for ele in to_replace:
    df['author'] = df['author'].apply(lambda x: x.replace(ele," "))

#Remove extra whitespace
df['author']=df['author'].apply(lambda x: (' '.join(x.split())).lower())

In [23]:
# Check for empty strings or short named authors
check = {i for i in range(len(df)-1) if len(df['author'][i]) <3}

[df[['title','author','url']].iloc[i] for i in check]

[title     Mr. Darcy Goes Overboard: A Tale of Tide & Pre...
 author                                                    j
 url       https://www.goodreads.com/book/show/10425347-m...
 Name: 14259, dtype: object,
 title                                              Shifters
 author                                                   dp
 url       https://www.goodreads.com/book/show/18292964-s...
 Name: 15087, dtype: object]

After cleaning, there are two who's author is just one or two letters. Going into the URL we can get the author's name and then rename the cells

In [24]:
df.loc[14259,'author'] = 'belinda roberts'
df.loc[15087,'author'] = 'douglas pershing'

### Cleaning the column language

In [25]:
df['language'].unique()

array(['English', 'Dutch', 'French', 'Indonesian', 'German', 'Portuguese',
       'Spanish, Castilian', 'Finnish', 'Norwegian', 'Spanish', 'Polish',
       'Japanese', 'Italian', 'Russian', 'Swedish', 'Ukrainian',
       'Bulgarian', 'Tamil', 'Danish', 'Afrikaans',
       'Norwegian Nynorsk; Nynorsk, Norwegian', 'Filipino; Pilipino',
       'Greek, Modern (1453–)', 'Greek, Modern (1453-)', 'Hungarian',
       'Estonian', 'Bokmål, Norwegian; Norwegian Bokmål', 'Scots',
       'Dutch, Flemish', 'Romanian', 'Persian', 'Czech', 'Arabic',
       'Hindi', 'Catalan; Valencian', 'Tagalog', 'Malay', 'Latvian',
       'Vietnamese', 'Dutch, Middle (ca.1050-1350)', 'Turkish', 'Korean',
       'Slovenian', 'Multiple languages', 'Chinese', 'Xhosa', 'Basque',
       'Lithuanian', 'Malayalam', 'Hebrew', 'Esperanto', 'Georgian',
       'Urdu', 'Albanian', 'Marathi', 'Maltese', 'Serbian', 'Croatian',
       'Greek, Ancient (to 1453)', 'Catalan, Valencian', 'Galician',
       'English, Middle (1100-1500)

There are a few values that are tagged with extra info like English, Middle (1100-1500) or more then one version of the language attached: 'Romanian, Moldavian, Moldovan'. Having this granularity only for certain rows would skrew the data. So we'll keep only the first language given in the cell and drop the rest.

In [26]:
# Remove all non alphabetical characters
df['language']=df['language'].apply(lambda x: re.sub('[^A-Za-z]+', ' ', str(x)))

# keep only the first text
# Bokm is a unique case as there's Bokm Norwegian and Norwegian
df['language']=df['language'].apply(lambda x: 'Norwegian' if x[:4]=='Bokm' else x[:scf.blank_index(x)].strip())

In [27]:
#save cleaned dataset for EDA and ML
df.to_csv('./cleaned_datasets/books_clean.csv',index=False)